<a href="https://colab.research.google.com/github/mostafa-ja/LLM_from_scratch/blob/main/load_small_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install transformers
!pip install tiktoken

ERROR: Could not find a version that satisfies the requirement data_common (from versions: none)
ERROR: No matching distribution found for data_common


In [5]:
import argparse, os
import tiktoken
from transformers import AutoTokenizer #LLaMA 3 tokenizer

`encode_ordinary(s)` is a method from the **`tiktoken`** library used specifically for GPT-style tokenization.

---

> 🔍 What It Does

```python
enc = tiktoken.get_encoding("gpt2")
enc.encode_ordinary(s)
```

This encodes the input string `s` into a list of **token IDs**, but **without** adding any *special tokens* like:

* `<|endoftext|>` (EOT)
* Padding or BOS/EOS tokens (if used in other contexts)

---

> ✅ Example

```python
enc = tiktoken.get_encoding("gpt2")
enc.encode_ordinary("Hello world")
# Output: [15496, 995]
```

* `15496` = "Hello"
* `995` = " world" (note the space)

Now compare that with a tokenizer that **adds special tokens**:

```python
enc.encode("Hello world")
# Output: [15496, 995, 50256]
```

* `50256` = `<|endoftext|>` (automatically appended)

So:

* `encode_ordinary(s)` → *pure* tokenization of the text, no extras.
* `encode(s)` → might add special tokens (like EOT) depending on settings.

---

> 🔧 Why Use `encode_ordinary`?

In many training pipelines, especially when you're manually managing formatting (like inserting EOTs), you want **full control**, so `encode_ordinary` is preferred.




Here's a **minimal example** showing how token sequences differ **with** and **without** the EOT token.

---

> ⚙ Setup

We'll compare tokenized output for two short "documents" using the GPT-2 tokenizer via `tiktoken`:

```python
import tiktoken

enc = tiktoken.get_encoding("gpt2")
eot = enc._special_tokens['<|endoftext|>']

doc1 = "To be or not to be."
doc2 = "That is the question."
```

---

> 🧪 1. **Without EOT**

```python
tokens = enc.encode_ordinary(doc1) + enc.encode_ordinary(doc2)
print(tokens)
```

**Output:**

```
[539, 389, 329, 703, 539, 13, 1804, 318, 262, 1123, 13]
```

* This is just the two texts glued together.
* The model may treat them as part of **one coherent sentence**, even though they're conceptually separate.

---

> 🧪 2. **With EOT Between Documents**

```python
tokens = [eot] + enc.encode_ordinary(doc1) + [eot] + enc.encode_ordinary(doc2)
print(tokens)
```

**Output (example):**

```
[50256, 539, 389, 329, 703, 539, 13, 50256, 1804, 318, 262, 1123, 13]
```

Here:

* `50256` is the EOT token for GPT-2.
* The model now sees two **clearly separated** text segments:

  * `[EOT] To be or not to be.`
  * `[EOT] That is the question.`

This helps the model reset context and avoid blending unrelated documents.

---

> 🧠 Why It Matters

Without EOT:

* The model may predict `"That"` as a continuation of `"To be or not to be."`

With EOT:

* The model is less likely to do that, since it sees a clear **document boundary**.

---



> 🤔 Why is the EOT placed **at the beginning** of each document (instead of at the end)?

```python
tokens.append(eot)  # prepend EOT
tokens.extend(encode(spad))  # then add the actual text
```

> 🧠 The Reason: **Training with autoregressive language models (like GPT-2)**

In **causal language modeling**, models are trained to predict the **next token** given all previous tokens.

That means:

* At training time, each token is predicted based on everything **to its left**.
* So, what you **put before** a sentence matters most.

---

> ✅ Why EOT at the Beginning Works

Putting an EOT **before** a new sentence acts as a **context reset**:

* It signals: “We’re starting a fresh, new document now.”
* The model learns: **"Whenever I see `<|endoftext|>`, forget what came before — I'm starting something new."**

> ❌ Why EOT at the End Isn't Enough

If you only put EOT **after** the sentence, it comes **too late**:

* The model would process the whole sentence **without knowing** it’s a new doc.
* It can carry over context from previous sentences — especially in long sequences.

---


In [ ]:
def get_tokenizer(model_desc):
    """Returns tokenizer function and end-of-text token based on model."""
    if model_desc == "gpt-2":
        enc = tiktoken.get_encoding("gpt2")
        encode = lambda s: enc.encode_ordinary(s)
        eot_token = enc._special_tokens['<|endoftext|>']
    elif model_desc == "llama-3":
        tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3.1-8B")
        encode = lambda s: tokenizer.encode(s, add_special_tokens=False, verbose=False, split_special_tokens=True)
        eot_token = tokenizer.encode('')[0]  # Adds EOT token by default
    else:
        raise ValueError(f"Unknown model descriptor: {model_desc}")
    return encode, eot_token

def process_text_file(filepath, encode, eot_token, train_split=0.9):
    """Reads text, tokenizes with EOT prepended to each section, and splits into train/val."""
    with open(filepath, 'r', encoding='utf-8') as f:
        text = f.read()

    # Splits the text into chunks wherever there are double newlines (\n\n), treating each as a separate "document" and Remove those \n\n separators.
    sections = text.split("\n\n")
    tokens = []

    for i, section in enumerate(sections):
        tokens.append(eot_token)  # Prepend EOT to mark document start
        padded_section = section + "\n\n" if i != len(sections) - 1 else section
        tokens.extend(encode(padded_section))

    split_idx = int(train_split * len(tokens))
    train_tokens = tokens[:split_idx]
    val_tokens = tokens[split_idx:]
    return train_tokens, val_tokens


In [ ]:
os.makedirs('dataset', exist_ok=True)
!wget -O dataset/tiny_shakespeare.txt https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

In [ ]:
model_desc = "gpt-2"
data_path = os.path.join('dataset', "tiny_shakespeare.txt")

encode_fn, eot = get_tokenizer(model_desc)
train_tokens, val_tokens = process_text_file(data_path, encode_fn, eot)

In [ ]:
import torch

class DataLoaderLite:
    def __init__(self, tokens, batch_size, block_size):
        self.tokens = torch.tensor(tokens, dtype=torch.long)
        self.batch_size = batch_size
        self.block_size = block_size

        self.total_tokens = len(self.tokens)
        self.tokens_per_epoch = (self.total_tokens - 1) // (batch_size * block_size)
        assert self.tokens_per_epoch > 0, "Tokens per epoch must be positive"

        # Truncate to a clean multiple
        usable_tokens = self.tokens_per_epoch * batch_size * block_size + 1
        self.tokens = self.tokens[:usable_tokens]

        self.current_position = 0
        print(f"1 epoch in data loader = {self.tokens_per_epoch} batches")

    def next_batch(self):
        B, T = self.batch_size, self.block_size
        start = self.current_position
        end = start + B * T + 1

        if end > len(self.tokens):
            self.current_position = 0
            start, end = 0, B * T + 1

        buf = self.tokens[start:end]
        x = buf[:-1].view(B, T)
        y = buf[1:].view(B, T)

        self.current_position += B * T
        return x, y

    def __next__(self):
        if self.current_position + self.batch_size * self.block_size + 1 > len(self.tokens):
            print('Next epoch ....')
        return self.next_batch()
